In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 16
### Team Member Names: Stephen Chen, Bhavya Shah, Alex Liu
### Team Strategy Chosen: RISKY?__________(RISKY OR SAFE)

In [2]:
# Importing csv file with tickers and reformatting dataframe
ticker_list = pd.read_csv("Tickers.csv")
add_columns = pd.DataFrame({ticker_list.columns[0]:ticker_list.columns[0]}, index=[len(ticker_list)])
ticker_list = ticker_list.append(add_columns)
ticker_list.columns=['Tickers']

In [3]:
ticker_list

,Tickers
0,ABBV
1,ABT
2,ACN
3,AGN
4,AIG
...,...
57,UNH
58,UNP
59,UPS
60,USB


In [ ]:
# identifying American stocks, converting tickers to American (if possible)
for i in range(0, len(ticker_list)):
    ticker = str(ticker_list.iloc[i, 0])
    
    # finds the period in the stock tickers
    get_position = ticker.find('.')
    
    # if there is a period in the string (meaning that it's not an American stock)
    if (get_position != -1):
        ticker_list.iloc[i, 0] = ticker[:get_position]

In [4]:
sector_list = pd.DataFrame(columns = ['Sectors'])

def get_deviation(ticker):
    stock_ticker = ticker
    stock_hist = stock_ticker.history(start='2019-01-01', end='2021-07-01')
    return (stock_hist['Close'].resample('MS').ffill().pct_change()*100).std()

i = 0

for i in range (len(ticker_list)):
    stock = yf.Ticker(ticker_list.iloc[i, 0])
    sector = stock.info.get('sector')
    add_data = pd.DataFrame({'Sectors':sector,
                             'Standard Deviation': get_deviation(stock)},index=[i])
    sector_list = sector_list.append(add_data)



- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: None
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted


In [6]:
ticker_list = pd.concat([ticker_list, sector_list], join='inner',axis=1)

In [7]:
ticker_list = ticker_list[ticker_list.Sectors.notnull()]
ticker_list.set_index('Sectors', inplace=True)

In [8]:
ticker_list

,Tickers,Standard Deviation
Sectors,,
Healthcare,ABBV,15.531777
Healthcare,ABT,16.292553
Technology,ACN,40.702581
Financial Services,AIG,9.304261
Consumer Cyclical,AMZN,670.251723
Financial Services,AXP,19.733866
Industrials,BA,86.688335
Financial Services,BAC,5.489330
Healthcare,BIIB,37.363508


In [ ]:
# placeholder volume column
ticker_list["Volume"] = None

# loop to calculate the average daily volume of each stock
for i in range (0, len(ticker_list)):
    
    # getting the number of days between June 2, 2021 and October 22, 2021
    num_days = int(str(pd.to_datetime('2021-07-02') - pd.to_datetime('2021-10-22'))[1:4])
    
    # getting each ticker's information
    ticker = yf.Ticker(ticker_list.iloc[i, 0])
    ticker_hist = ticker.history(start="2021-07-02", end="2021-10-22")
    ticker_volume = ticker_hist['Volume'].sum()/num_days
    ticker_list.iloc[i, -1] = ticker_volume
    
# filtering for the tickers that fit our requirement
ticker_list = ticker_list[(ticker_list["Volume"] > 10000)]

In [9]:
def sort_list (industry_list, industry):
    industry_list = ticker_list.filter(like = industry, axis=0)
    industry_list = industry_list.sort_values(by=['Standard Deviation'], ascending = False)
    return industry_list
    

In [10]:
energy_list = sort_list (ticker_list, 'Energy')
energy_list

,Tickers,Standard Deviation
Sectors,,
Energy,OXY,16.148199
Energy,COP,10.335786
Energy,SLB,8.584740
Energy,KMI,2.314005


In [20]:
financial_list = sort_list (ticker_list, 'Financial Services') 
financial_list

,Tickers,Standard Deviation
Sectors,,
Financial Services,BLK,143.352853
Financial Services,PYPL,62.536603
Financial Services,COF,25.922625
Financial Services,JPM,21.602003
Financial Services,AXP,19.733866
Financial Services,MS,16.719687
Financial Services,C,10.746483
Financial Services,AIG,9.304261
Financial Services,TD.TO,8.465006


In [11]:
tech_list = sort_list (ticker_list, 'Technology')
tech_list

,Tickers,Standard Deviation
Sectors,,
Technology,SHOP.TO,501.447116
Technology,MSFT,47.538964
Technology,ACN,40.702581
Technology,AAPL,33.223412
Technology,QCOM,32.163147
Technology,TXN,28.827644
Technology,ORCL,8.567385
Technology,CSCO,4.891558


In [12]:
healthcare_list = sort_list (ticker_list, 'Healthcare')
healthcare_list

,Tickers,Standard Deviation
Sectors,,
Healthcare,UNH,53.881059
Healthcare,BIIB,37.363508
Healthcare,LLY,32.652375
Healthcare,ABT,16.292553
Healthcare,ABBV,15.531777
Healthcare,CVS,9.262210
Healthcare,BMY,7.652742
Healthcare,MRK,3.731006
Healthcare,PFE,2.628826


In [13]:
estate_list = sort_list (ticker_list, 'Real Estate')
estate_list

,Tickers,Standard Deviation
Sectors,,
Real Estate,SPG,36.083989


In [14]:
industry_list = sort_list (ticker_list, 'Industrials')
industry_list

,Tickers,Standard Deviation
Sectors,,
Industrials,BA,86.688335
Industrials,CAT,38.676574
Industrials,LMT,35.805188
Industrials,UPS,35.668689
Industrials,UNP,24.830200


In [15]:
utilities_list = sort_list(ticker_list, 'Utilities')
utilities_list

,Tickers,Standard Deviation
Sectors,,
Utilities,NEE,11.550032
Utilities,SO,5.939180


In [16]:
communication_list = sort_list(ticker_list,'Communication Services')
communication_list

,Tickers,Standard Deviation
Sectors,,
Communication Services,GOOG,397.476396
Communication Services,CMCSA,6.359033
Communication Services,T,2.680472


In [17]:
materials_list = sort_list(ticker_list,'Basic Materials')
materials_list

,Tickers,Standard Deviation
Sectors,,


In [18]:
consumer_list = sort_list(ticker_list,'Consumer Defensive')
consumer_list

,Tickers,Standard Deviation
Sectors,,
Consumer Defensive,COST,52.244018
Consumer Defensive,TGT,46.548591
Consumer Defensive,PG,14.402693
Consumer Defensive,PEP,11.008535
Consumer Defensive,PM,8.572384
Consumer Defensive,CL,6.873169
Consumer Defensive,MO,4.324744
Consumer Defensive,KO,4.003186


In [21]:
ticker_list = pd.concat([energy_list, financial_list, tech_list, healthcare_list, estate_list,industry_list, utilities_list,communication_list,
                         materials_list, consumer_list])

In [22]:
ticker_list = ticker_list[:10]
ticker_list

,Tickers,Standard Deviation
Sectors,,
Energy,OXY,16.148199
Energy,COP,10.335786
Energy,SLB,8.584740
Energy,KMI,2.314005
Financial Services,BLK,143.352853
Financial Services,PYPL,62.536603
Financial Services,COF,25.922625
Financial Services,JPM,21.602003
Financial Services,AXP,19.733866


In [23]:
def get_returns (ticker):
    stock = yf.Ticker(ticker)
    start_date = '2019-01-01'
    end_date = '2021-11-01'
    history = stock.history(start=start_date, end=end_date)
    prices = pd.DataFrame({ticker: history['Close']})
    prices = prices.resample('MS').ffill()
    prices = prices.pct_change()
    return prices

get_returns ('OXY')

#return_list = get_returns(ticker_list.iloc[0,0])


,OXY
Date,
2019-01-01,NaN
2019-02-01,NaN
2019-03-01,-0.007202
2019-04-01,0.006464
2019-05-01,-0.144388
2019-06-01,-0.134134
2019-07-01,0.016367
2019-08-01,0.068100
2019-09-01,-0.182246


In [24]:
return_list = get_returns(ticker_list.iloc[0,0])

In [25]:
i = 1
for i in range (len(ticker_list)):
    ticker = ticker_list.iloc[i,0]
    add_returns = get_returns (ticker)
    return_list = pd.concat([return_list, add_returns], join = 'inner', axis = 1)

In [26]:
return_list = return_list.iloc[: , 1:]

In [27]:
return_list

,OXY,COP,SLB,KMI,BLK,PYPL,COF,JPM,AXP,MS
Date,,,,,,,,,,
2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-01,-0.007201,0.008197,0.025879,0.075749,0.064682,0.097656,0.047090,0.005294,0.056666,0.016260
2019-04-01,0.006464,-0.017989,-0.009515,0.015198,-0.004612,0.068320,0.002611,0.002011,0.025712,0.024235
2019-05-01,-0.144388,-0.074162,-0.075737,-0.010224,0.092931,0.051066,0.095881,0.108956,0.051336,0.099698
2019-06-01,-0.134134,-0.054513,-0.161470,0.018377,-0.132678,-0.010727,-0.068346,-0.079889,-0.019740,-0.144629
2019-07-01,0.016367,0.055801,0.149665,0.046616,0.146764,0.048109,0.071853,0.072858,0.089007,0.080118
2019-08-01,0.068100,-0.050621,-0.020600,0.006781,-0.044362,-0.036773,-0.008040,0.000524,-0.003538,-0.015591
2019-09-01,-0.182246,-0.112585,-0.157881,-0.024073,-0.065029,-0.015794,-0.047085,-0.027271,-0.029978,-0.033543


In [28]:
corr = return_list.corr()
corr

,OXY,COP,SLB,KMI,BLK,PYPL,COF,JPM,AXP,MS
OXY,1.000000,0.864916,0.837366,0.768746,0.458643,0.548201,0.665041,0.714820,0.737800,0.704191
COP,0.864916,1.000000,0.919729,0.824669,0.508314,0.551657,0.775014,0.810536,0.841218,0.738719
SLB,0.837366,0.919729,1.000000,0.847104,0.663744,0.612465,0.825672,0.896199,0.848938,0.859640
KMI,0.768746,0.824669,0.847104,1.000000,0.578690,0.517712,0.813793,0.774800,0.830655,0.723309
BLK,0.458643,0.508314,0.663744,0.578690,1.000000,0.622527,0.668361,0.710484,0.643570,0.836869
PYPL,0.548201,0.551657,0.612465,0.517712,0.622527,1.000000,0.615333,0.500448,0.616488,0.727705
COF,0.665041,0.775014,0.825672,0.813793,0.668361,0.615333,1.000000,0.875183,0.837570,0.828841
JPM,0.714820,0.810536,0.896199,0.774800,0.710484,0.500448,0.875183,1.000000,0.835934,0.887342
AXP,0.737800,0.841218,0.848938,0.830655,0.643570,0.616488,0.837570,0.835934,1.000000,0.839531
MS,0.704191,0.738719,0.859640,0.723309,0.836869,0.727705,0.828841,0.887342,0.839531,1.000000


In [29]:
highest_corr = pd.DataFrame({'test':corr[corr.columns[0]].nlargest(2)})
highest_corr

,test
OXY,1.000000
COP,0.864916


In [30]:
def stock_df (ticker, value, num):
    myhistory = (yf.Ticker(ticker).history(start='2021-05-19', end='2021-11-30', interval= '1d'))
    data= {'Ticker': [ticker],
           'Price': [myhistory.loc['2021-06-18', 'Close']],
           'Shares': value/myhistory.loc['2021-06-18', 'Close'],
           'Values': [value], 'Weight (Percent)': [value/1000]}
    grades= pd.DataFrame(data,index=[num])
    return grades

stock1 = stock_df (highest_corr.index[0], 35000, 1)
stock2 = stock_df (highest_corr.index[1], 25000, 2)

In [31]:
ticker_list = ticker_list[ticker_list.Tickers != highest_corr.index[0]]

In [32]:
ticker_list = ticker_list[ticker_list.Tickers != highest_corr.index[1]]

In [33]:
ticker_list

,Tickers,Standard Deviation
Sectors,,
Energy,SLB,8.584740
Energy,KMI,2.314005
Financial Services,BLK,143.352853
Financial Services,PYPL,62.536603
Financial Services,COF,25.922625
Financial Services,JPM,21.602003
Financial Services,AXP,19.733866
Financial Services,MS,16.719687


In [37]:
FinalPortfolio = stock1.append(stock2)

i = 0
for i in range (8):
    add_stock = stock_df(ticker_list.iloc[i,0], 5000, i+3)
    FinalPortfolio = FinalPortfolio.append(add_stock)
    
total = pd.DataFrame({'Ticker': 'N/A',
                      'Price': 'N/A',
                      'Shares': 'N/A',
                      'Values': sum(FinalPortfolio.Values),
                      'Weight (Percent)': sum(FinalPortfolio['Weight (Percent)'])}, index=[11])



,Ticker,Price,Shares,Values,Weight (Percent)
11,N/A,N/A,N/A,100000,100.0


In [38]:
FinalPortfolio = FinalPortfolio.append(total)

In [39]:
FinalPortfolio

,Ticker,Price,Shares,Values,Weight (Percent)
1,OXY,27.978764,1250.948774,35000,35.0
2,COP,56.752529,440.509002,25000,25.0
3,SLB,31.151695,160.504909,5000,5.0
4,KMI,17.122942,292.005895,5000,5.0
5,BLK,837.127441,5.972806,5000,5.0
6,PYPL,283.380005,17.644152,5000,5.0
7,COF,148.348282,33.704469,5000,5.0
8,JPM,146.191544,34.201705,5000,5.0
9,AXP,157.367157,31.77283,5000,5.0
10,MS,82.903671,60.31096,5000,5.0


In [40]:
FinalPortfolio.drop(FinalPortfolio.tail(1).index,inplace=True)

In [50]:
FinalPortfolio.reset_index(inplace=True)

In [51]:
FinalPortfolio

,index,Ticker,Price,Shares,Values,Weight (Percent)
0,1,OXY,27.978764,1250.948774,35000,35.0
1,2,COP,56.752529,440.509002,25000,25.0
2,3,SLB,31.151695,160.504909,5000,5.0
3,4,KMI,17.122942,292.005895,5000,5.0
4,5,BLK,837.127441,5.972806,5000,5.0
5,6,PYPL,283.380005,17.644152,5000,5.0
6,7,COF,148.348282,33.704469,5000,5.0
7,8,JPM,146.191544,34.201705,5000,5.0
8,9,AXP,157.367157,31.77283,5000,5.0
9,10,MS,82.903671,60.31096,5000,5.0


In [55]:
Stocks = pd.concat([FinalPortfolio['index'], FinalPortfolio['Ticker'], FinalPortfolio['Shares']], join='inner',axis=1)
Stocks.columns=['','Ticker','Shares']

In [56]:
Stocks

,,Ticker,Shares
0,1,OXY,1250.948774
1,2,COP,440.509002
2,3,SLB,160.504909
3,4,KMI,292.005895
4,5,BLK,5.972806
5,6,PYPL,17.644152
6,7,COF,33.704469
7,8,JPM,34.201705
8,9,AXP,31.77283
9,10,MS,60.31096


In [57]:
Stocks.to_csv('Stocks_Group_16.csv', encoding='utf-8', index=False)

In [ ]:
# getting the monthly returns of each sector
def find_returns(df):
    
    # sectors without any tickers inside of them
    if (len(df) == 0):
        return None
    
    # sectors with tickers inside of them
    else:
        
        # creating a temporary dataframe (with the first value)
        init_frame = pd.DataFrame(columns=['Close Prices'])
        init_ticker = yf.Ticker(df.iloc[0,0])
        init_hist = init_ticker.history(start="2019-01-01", end="2021-01-01")
        init_close = init_hist['Close']
        init_frame['Close Prices'] = init_close
        
        if (len(df) == 1):
            init_frame['Monthly Returns'] = init_frame['Close Prices'].pct_change()
            return init_frame['Monthly Returns']
    
        else:
            # looping through the sectors to get the monthly returns
            for i in range (1, len(df)-1):
                ticker = yf.Ticker(df.iloc[i, 0])
                ticker_hist = ticker.history(start="2019-01-01", end="2021-01-01")
                ticker_close = ticker_hist['Close']
            
                # adding the close prices of each stock
                init_frame['Close Prices'] = init_frame['Close Prices'] + ticker_close
        
                # calculating the monthly returns
                init_frame['Monthly Returns'] = init_frame['Close Prices'].pct_change()
        
                # returning the monthly returns
                return init_frame['Monthly Returns']
    
    return 0

In [ ]:
correlation = pd.DataFrame()
correlation['Energy Monthly Returns (%)'] = get_returns(energy_list)
correlation['Financial Services Monthly Returns (%)'] = get_returns(financial_list)
correlation['Technology Monthly Returns (%)'] = get_returns(tech_list)
correlation['Healthcare Monthly Returns (%)'] = get_returns(healthcare_list)
correlation['Real Estate Monthly Returns (%)'] = get_returns(estate_list)
correlation['Industrials Monthly Returns (%)'] = get_returns(industry_list)
correlation['Utilities Monthly Returns (%)'] = get_returns(utilities_list)
correlation['Communication Services Monthly Returns (%)'] = get_returns(communication_list)
correlation['Materials Monthly Returns (%)'] = get_returns(materials_list)
correlation['Consumer Defensive Monthly Returns (%)'] = get_returns(consumer_list)

print(correlation.corr())

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.